In [1]:
'''导入相关库'''


import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader,TensorDataset
import torchvision
import torchvision.transforms as transforms

In [2]:
''' 
确定数据， 确定超参数！
'''
lr = 0.15
gamma = 0.1
epochs = 50
batch_size = 128

In [3]:
mnist = torchvision.datasets.FashionMNIST(root = './MNIST_data',
                                          download= False
                                          ,train=True,
                                          transform=transforms.ToTensor()
                                          )

In [4]:
# 
batchdata = DataLoader(mnist, batch_size,shuffle=True)

In [5]:
for  x, y in batchdata:
    print(x.shape)
    print(y.shape)
    break

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [6]:
#问一下一共有多少个元素呢？
mnist.data[0].numel()


784

In [7]:
input_ = mnist.data[0].numel()
output_ = len(mnist.targets.unique())
print(output_)

10


In [8]:
#定义神经网路的架构
class Model(nn.Module):
    def __init__(self,in_features=10,out_features = 2):
        super().__init__()
        self.linear1 = nn.Linear(in_features,128,bias=False)
        self.output = nn.Linear(128,out_features,bias=False)

    def forward(self,x):
        x = x.view(-1,28*28)
        sigma1 = torch.relu(self.linear1(x))
        sigma2 = F.log_softmax(self.output(sigma1),dim=1)
        # 将sigma1输入self.output中得到一个输出，这个输出将作为最终的输出sigma2。所以，self.output中存储的就是一个线性层，也就是用于将sigma1变换为最终输出的权重矩阵。
        return sigma2
''' 

在深度学习中，.view(-1)的作用是将一个多维张量的形状展平成一维，以便于输入到全连接层或
其他处理一维数据的模型中。具体来说，-1表示自动计算维度大小，可以根据张量的总元素个数和已知的其
他维度大小推导出该维度的大小。例如，如果一个3x4x5的张量调用.view(-1)，则会得到一个60维的一维张量
。这样做有助于减少模型中的参数数量和计算负担，同时防止过拟合。
'''


' \n\n在深度学习中，.view(-1)的作用是将一个多维张量的形状展平成一维，以便于输入到全连接层或\n其他处理一维数据的模型中。具体来说，-1表示自动计算维度大小，可以根据张量的总元素个数和已知的其\n他维度大小推导出该维度的大小。例如，如果一个3x4x5的张量调用.view(-1)，则会得到一个60维的一维张量\n。这样做有助于减少模型中的参数数量和计算负担，同时防止过拟合。\n'

In [10]:
#定义损失函数和优化算法以及梯度下降的内容
#训练函数
def fit(net,batchdata,lr =0.01, gamma= 0.8, epochs = 20):
    criterion = nn.NLLLoss()
    opt = optim.SGD(net.parameters(),lr = lr, momentum=gamma)
    sample = 0#循环前无样本
    correct = 0#循环开始前正确的为0
    for epoch in range(epochs):
        for batch_idx,(x,y) in enumerate(batchdata):
            y = y.view(x.shape[0])# 降低维度
            sigma = net.forward(x)
            loss = criterion(sigma,y)
            loss.backward()
            opt.step()
            opt.zero_grad()

            sample += x.shape[0]
            #求解准确率，全部判断正确的样本/已经看过的样本！
            yhat = torch.max(sigma,1)[1] #torch.max函数中结果索引为1的标签！
            correct += torch.sum( yhat == y )


            if (batch_idx + 1 ) %  5 == 0:
                
            # 分子 已经看过 分母 现有的epoch的一共多少！
                print("Epoch{}:[{}/{}({:.0f}%)],Loss:{:.6f},Accuracy:{:.3f}".format(
                                              epoch+1
                                              ,sample
                                              ,epochs*len(batchdata.dataset)
                                              ,100*sample/(epochs*len(batchdata.dataset))
                                              ,loss.data.item()
                                              ,float(100*correct/sample)
                                              ))
                



In [11]:
#模型的训练与评估
torch.manual_seed(1420)
net = Model(in_features=input_,out_features=output_)
fit(net,batchdata,lr=lr,epochs=epochs,gamma=gamma)


Epoch1:[640/3000000(0%)],Loss:2.057418,Accuracy:26.250
Epoch1:[1280/3000000(0%)],Loss:1.785232,Accuracy:35.625
Epoch1:[1920/3000000(0%)],Loss:1.590194,Accuracy:40.469
Epoch1:[2560/3000000(0%)],Loss:1.301049,Accuracy:45.391
Epoch1:[3200/3000000(0%)],Loss:1.073391,Accuracy:48.750
Epoch1:[3840/3000000(0%)],Loss:0.961707,Accuracy:51.536
Epoch1:[4480/3000000(0%)],Loss:0.910317,Accuracy:52.946
Epoch1:[5120/3000000(0%)],Loss:1.091040,Accuracy:54.004
Epoch1:[5760/3000000(0%)],Loss:0.880227,Accuracy:55.191
Epoch1:[6400/3000000(0%)],Loss:0.765669,Accuracy:56.406
Epoch1:[7040/3000000(0%)],Loss:0.801039,Accuracy:57.614
Epoch1:[7680/3000000(0%)],Loss:0.747113,Accuracy:58.438
Epoch1:[8320/3000000(0%)],Loss:0.881608,Accuracy:59.171
Epoch1:[8960/3000000(0%)],Loss:0.849836,Accuracy:59.933
Epoch1:[9600/3000000(0%)],Loss:0.699564,Accuracy:60.708
Epoch1:[10240/3000000(0%)],Loss:0.752064,Accuracy:61.445
Epoch1:[10880/3000000(0%)],Loss:0.639736,Accuracy:62.261
Epoch1:[11520/3000000(0%)],Loss:0.772413,Accura